# Modulos

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
import re
import os
import openpyxl

# Añadir la carpeta raíz del proyecto a sys.path
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath("D:\Proyectos Python\capresoca-data-automation"))

from src.file_loader import cargar_maestros_ADRES

# Rutas y variables

In [2]:
R_Salida = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Consulta SISBEN\08_AGOSTO"
fecha = datetime.now().strftime("%d-%m-%Y")

R_MS_EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0015082025.TXT"
R_MS_EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0015082025.TXT"
R_MS_SIE = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_08_19.csv"



# Dataframes

In [3]:
# Cargar archivos que sirven para validación
df_MS_SIE = pd.read_csv(R_MS_SIE, delimiter=';', dtype=str, encoding='ansi')

# Cargar y combinar los maestros
df_maestro_ADRES = cargar_maestros_ADRES(R_MS_EPS025, R_MS_EPSC25)

# Limpieza de datos

In [4]:
df_MS_SIE.columns

Index(['tipo_documento', 'numero_identificacion', 'primer_apellido',
       'segundo_apellido', 'primer_nombre', 'segundo_nombre',
       'fecha_nacimiento', 'genero', 'municipio', 'estado', 'regimen',
       'direccion', 'celular', 'telefono_1', 'telefono_2',
       'correo_electronico', 'tipo_afiliado', 'parentesco_codificado',
       'tipo_documento_padre', 'numero_documento_padre', 'primer_nombre_padre',
       'segundo_nombre_padre', 'primer_apellido_padre',
       'segundo_apellido_padre', 'estado_traslado', 'nivel_sisben', 'ficha',
       'subgrupo', 'grupo_poblacional', 'fecha_sgsss', 'primer_nombre_bdua',
       'segundo_nombre_bdua', 'primer_apellido_bdua', 'segundo_apellido_bdua',
       'discapacidad', 'ips_primaria', 'municipio_nacimiento', 'categoria_ibc',
       'serial_fosyga', 'Unnamed: 39'],
      dtype='object')

In [5]:
df_MS_SIE['estado_traslado'].unique()

array(['No Aplica', 'Pendiente Ingreso Traslado RS',
       'Pendiente Solicitud Retiro Traslado RS',
       'Pendiente Solicitud Retiro Traslado RC', 'Pendiente Ingreso MS',
       'Pendiente Ingreso Traslado RC', 'Pendiente Ingreso MC'],
      dtype=object)

In [6]:
# Filtrar los registros según los valores requeridos en 'estado_traslado'
estados_filtrar = [
    'Pendiente Ingreso Traslado RS',
    'Pendiente Ingreso MS',
    'Pendiente Ingreso Traslado RC',
    'Pendiente Ingreso MC'
]

# Contar registros antes del filtrado
registros_antes = len(df_MS_SIE)

# Realizar el filtrado
df_MS_SIE = df_MS_SIE[df_MS_SIE['estado_traslado'].isin(estados_filtrar)]

# Contar registros después del filtrado
registros_despues = len(df_MS_SIE)

print(f"Registros antes del filtrado: {registros_antes}")
print(f"Registros después del filtrado: {registros_despues}")

Registros antes del filtrado: 224953
Registros después del filtrado: 2068


In [7]:
df_MS_SIE = df_MS_SIE[['tipo_documento', 'numero_identificacion', 'fecha_nacimiento']]

In [8]:
df_MS_SIE.columns

Index(['tipo_documento', 'numero_identificacion', 'fecha_nacimiento'], dtype='object')

In [9]:
df_maestro_ADRES.columns

Index(['AFL_ID', 'ENT_ID', 'TPS_IDN_ID_CF', 'HST_IDN_NUMERO_IDENTIFICACION_CF',
       'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO',
       'AFL_SEGUNDO_APELLIDO', 'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
       'AFL_FECHA_NACIMIENTO', 'TPS_GNR_ID', 'AFL_PAIS_NACIMIENTO',
       'AFL_MUNICIPIO_NACIMIENTO', 'AFL_NACIONALIDAD',
       'AFL_SEXO_IDENTIFICACION', 'AFL_DISCAPACIDAD', 'TPS_AFL_ID',
       'TPS_PRN_ID', 'TPS_GRP_PBL_ID', 'TPS_NVL_SSB_ID', 'NUMEROFICHASISBEN',
       'TPS_CND_BNF_ID', 'DPR_ID', 'MNC_ID', 'ZNS_ID',
       'AFL_FECHA_AFILIACION_SGSSS', 'AFC_FECHA_INICIO', 'NUMERO CONTRATO',
       'FECHADE INICIO DEL CONTRATO', 'CNT_AFL_TPS_GRP_PBL_ID',
       'CNT_AFL_TPS_PRT_ETN_ID_2', 'TPS_MDL_SBS_ID', 'TPS_EST_AFL_ID',
       'CND_AFL_FECHA_INICIO', 'CND_AFL_FECHA_INICIO', 'GRP_FML_COTIZANTE_ID',
       'PORTABILIDAD', 'COD_IPS_P', 'MTDLG_G_P', 'SUB_SISBEN_IV',
       'MARCASISBENIV+MARCASISBENIII', 'CRUCE_BDEX_RNEC'],
      dtype='object')

In [10]:
df_maestro_ADRES['TPS_IDN_ID'].unique()

array(['CN', 'CC', 'RC', 'PT', 'TI', 'PE', 'CE', 'MS', 'SC', 'AS'],
      dtype=object)

In [11]:
# Definir los valores a excluir
excluir = ['CN', 'PE', 'AS', 'SC']

# Contar registros antes del filtrado
registros_antes = len(df_maestro_ADRES)

# Filtrar el DataFrame
df_maestro_ADRES = df_maestro_ADRES[~df_maestro_ADRES['TPS_IDN_ID'].isin(excluir)]

# Contar registros después del filtrado
registros_despues = len(df_maestro_ADRES)

print(f"Registros antes del filtrado: {registros_antes}")
print(f"Registros después del filtrado: {registros_despues}")

Registros antes del filtrado: 197547
Registros después del filtrado: 197193


In [12]:
df_maestro_ADRES['TPS_EST_AFL_ID'].unique()

array(['AF', 'RE', 'AC', 'SD', 'SM', 'PL'], dtype=object)

In [13]:
# Filtrar los registros según los valores requeridos en 'TPS_EST_AFL_ID'
estados_filtrar = [
    'AC',
    'RE',
    'SM',
    'PL'
]

# Contar registros antes del filtrado
registros_antes = len(df_maestro_ADRES)

# Realizar el filtrado
df_maestro_ADRES = df_maestro_ADRES[df_maestro_ADRES['TPS_EST_AFL_ID'].isin(estados_filtrar)]

# Contar registros después del filtrado
registros_despues = len(df_maestro_ADRES)

print(f"Registros antes del filtrado: {registros_antes}")
print(f"Registros después del filtrado: {registros_despues}")

Registros antes del filtrado: 197193
Registros después del filtrado: 179149


In [14]:
df_maestro_ADRES = df_maestro_ADRES[['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_FECHA_NACIMIENTO']]

In [15]:
df_maestro_ADRES.columns

Index(['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_FECHA_NACIMIENTO'], dtype='object')

In [16]:
# Renombrar las columnas de df_maestro_ADRES para que coincidan con df_MS_SIE
df_maestro_ADRES_ren = df_maestro_ADRES.rename(columns={
    'TPS_IDN_ID': 'tipo_documento',
    'HST_IDN_NUMERO_IDENTIFICACION': 'numero_identificacion',
    'AFL_FECHA_NACIMIENTO': 'fecha_nacimiento'
})

# Estandarizar la columna 'fecha_nacimiento' de df_MS_SIE a formato dd/mm/yyyy
df_MS_SIE['fecha_nacimiento'] = pd.to_datetime(df_MS_SIE['fecha_nacimiento'], format='%Y-%m-%d').dt.strftime('%d/%m/%Y')

# Unificar ambos dataframes
df_unificado = pd.concat([df_MS_SIE, df_maestro_ADRES_ren], ignore_index=True)

# Mostrar las primeras filas del dataframe unificado
df_unificado.head()

del df_MS_SIE, df_maestro_ADRES_ren, df_maestro_ADRES

In [17]:
# Convertir 'fecha_nacimiento' a datetime para calcular la edad
df_unificado['fecha_nacimiento_dt'] = pd.to_datetime(df_unificado['fecha_nacimiento'], format='%d/%m/%Y', errors='coerce')
hoy = pd.to_datetime(fecha, format='%d-%m-%Y')

# Calcular la edad
df_unificado['edad'] = (hoy - df_unificado['fecha_nacimiento_dt']).dt.days // 365

# Duplicar registros de 18 y 19 años con CC a TI
mask_cc = (df_unificado['edad'].isin([18, 19])) & (df_unificado['tipo_documento'] == 'CC')
df_cc = df_unificado[mask_cc].copy()
df_cc['tipo_documento'] = 'TI'

# Duplicar registros de 7 y 8 años con TI a RC
mask_ti = (df_unificado['edad'].isin([7, 8])) & (df_unificado['tipo_documento'] == 'TI')
df_ti = df_unificado[mask_ti].copy()
df_ti['tipo_documento'] = 'RC'

# Concatenar los duplicados al dataframe original
df_unificado_ext = pd.concat([df_unificado, df_cc, df_ti], ignore_index=True)

# Validar que la dimensión aumentó
print(f"Dimensión original: {df_unificado.shape[0]}")
print(f"Dimensión extendida: {df_unificado_ext.shape[0]}")

# Limpiar columnas auxiliares si no se requieren
df_unificado_ext = df_unificado_ext.drop(columns=['fecha_nacimiento_dt', 'edad'])

Dimensión original: 181217
Dimensión extendida: 192178


# Guardar dataframes

In [18]:
# Asegurar que la columna 'fecha_nacimiento' esté en formato dd/mm/yyyy
df_unificado_ext['fecha_nacimiento'] = pd.to_datetime(df_unificado_ext['fecha_nacimiento'], dayfirst=True, errors='coerce').dt.strftime('%d/%m/%Y')

# Construir el nombre del archivo con el formato requerido
nombre_archivo = hoy.strftime('%Y_%m_%d') + '.xlsx'
ruta_archivo = os.path.join(R_Salida, nombre_archivo)

# Guardar el DataFrame en Excel sin encabezados
df_unificado_ext.to_excel(ruta_archivo, index=False, header=False, engine='openpyxl')
 

In [19]:
# Cambiar la extensión del archivo exportado de .xlsx a .XLSX (mayúscula)
ruta_archivo_nuevo = os.path.splitext(ruta_archivo)[0] + '.XLSX'

# Renombrar el archivo si la extensión es diferente
if ruta_archivo.lower() != ruta_archivo_nuevo:
    os.rename(ruta_archivo, ruta_archivo_nuevo)
    print(f"Archivo renombrado a: {ruta_archivo_nuevo}")
else:
    print("La extensión ya está en mayúscula.")

Archivo renombrado a: C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Consulta SISBEN\08_AGOSTO\2025_08_19.XLSX
